In [3]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import matplotlib.pyplot as plt

### basic functions for pre-processing

In [7]:
'''
read in both data and dictionary files, intercept with columes that describe volume
params: path to both files
param: keyword for filter out area of interest
return: dataframe of patientID, VISCODE, exam date and volumes of all regions in dict 
'''
def read_csv(file, dictfile, keyword):
    df = pd.read_csv(file)
    dict_df = pd.read_csv(dictfile)

    #filter out rows only with Volume
    dict_df = dict_df[dict_df["TEXT"].str.contains(keyword, case=False, na=False)]
    STcodes = dict_df['FLDNAME'].values
    extra_cols = ['RID','VISCODE','EXAMDATE']
    column_needed = np.concatenate([extra_cols, STcodes])
    
    #cross reference df with dict_df
    df = df.loc[:,column_needed]
    return df

'''
create dir with dirname in current folder to save all correlation matrices
'''
def make_dir(dirname):
    parent_dir = os.getcwd()
    path = os.path.join(parent_dir, dirname)
    os.mkdir(path)
    
'''
get correlation matrix per patient for all structures in the data file w.r.t volume
params: df is the processed dataframe with all data, path is the path for output dir
return: no return, saves all corr-mat to file
'''
def get_corr_mat(df, path):
    # get patients
    patients = np.unique(df['RID'].values)

    #fill rows with NA values at patient level
    for p in patients:
        pat_df = df[df['RID'] == p]

        ##only take into account of patients with more than 3 rows
        ##since 2 rows produce matrix of 1 and -1, and 1 row produces NA
        if len(pat_df.index) > 2:
            #convert to datetime, then sort by time
            try:
                pat_df['EXAMDATE'] = pd.to_datetime(pat_df['EXAMDATE'], format='%m/%d/%y')
            except:
                pat_df['EXAMDATE'] = pd.to_datetime(pat_df['EXAMDATE'], format='%Y-%m-%d')
            pat_df = pat_df.sort_values(by=['EXAMDATE'])

            #drop columns with all NAs
            pat_df = pat_df.dropna(axis=1, how='all')
            ##filling NA values
            pat_df = pat_df.fillna(method='ffill')
            #drop unnecessary rows for correlation matrix
            pad_df = pat_df.drop(['RID','VISCODE','EXAMDATE'],axis=1)
            corr_mat = pad_df.corr()

            out_path = path + '/Patient_' + str(p) + '.csv'
            corr_mat.to_csv(out_path)
            
'''
get adjacency matrix from corr matrix, above threshold we say there is an edge
params: df is the processed dataframe with all data, threshold is the absolute value of 
which there is an edge between two structures
return: the adjacent matrix
'''
def get_adj_mat(df, threshold):
    #make adjacency matrix based on threshold
    adj_mat = np.where(abs(df) >= threshold, 1, 0)
    
    ## fill adj mat diagonals with 0 to make hollow
    np.fill_diagonal(adj_mat, 0)
    
    cols = df.columns.values
    adj_df = pd.DataFrame(adj_mat, columns=cols, index=cols)
    return adj_df

### example dataset from UCSF Longitudinal study, first create correlation matrices

In [8]:
#UCSF Longitutinal FreeSurfer
longi_df = pd.read_csv('dataset/UCSFFSL_02_01_16.csv')
longi_dict_df = pd.read_csv('dataset/UCSFFSL_DICT_11_01_13.csv')
#filter out rows only with Volume
longi_dict_df = longi_dict_df[longi_dict_df["TEXT"].str.contains("Volume", case=False, 
                                                                 na=False)]
longi_STcodes = longi_dict_df['FLDNAME'].values

longi_extra_cols = ['RID','VISCODE','EXAMDATE','VERSION']
longi_column_needed = np.concatenate([longi_extra_cols, longi_STcodes])
longi_df = longi_df.loc[:,longi_column_needed]

In [1]:
##create a directory to save all corr matrices in for longitudinal
logi_dirname = 'UCSF_longitudinal_corr'
os.mkdir(logi_dirname)

#drop columns with all NAs
longi_df = longi_df.dropna(axis=1, how='all')

# get patients
longi_patients = np.unique(longi_df['RID'].values)

#fill rows with NA values at patient level
for p in longi_patients:
    pat_df = longi_df[longi_df['RID'] == p]
    
    ##only take into account of patients with more than 3 rows
    ##since 2 rows produce matrix of 1 and -1, and 1 row produces NA
    ##will need to think about a better way
    if len(pat_df.index) > 2:
        #convert to datetime, then sort by time
        pat_df['EXAMDATE'] = pd.to_datetime(pat_df['EXAMDATE'], format='%Y-%m-%d')
        pat_df = pat_df.sort_values(by=['EXAMDATE'])
        pat_df = pat_df.fillna(method='ffill')
        #drop columns with all NAs
        pat_df = pat_df.dropna(axis=1, how='all')
        
        ##drop strcutures not interested in this study
        pat_df = pat_df.drop(['RID','VISCODE','VERSION','EXAMDATE','ST10CV',
                             'ST123CV','ST125SV','ST128SV','ST64CV',
                             'ST66SV','ST68SV','ST69SV'], axis=1) 
        corr_mat = pat_df.corr()

        logi_out_path = logi_dirname + '/Patient_' + str(p) + '.csv'
        corr_mat.to_csv(logi_out_path)

### create adjacency matrices

In [10]:
# read correlation matrices and convert to adjacency
corr_mats = os.listdir('UCSF_longitudinal_corr')
threshold = 0.8

make_dir('UCSF_longitudinal_adj')
for i in corr_mats:
    df = pd.read_csv('UCSF_longitudinal_corr/' + i, index_col=0)
    adjacency = get_adj_mat(df, 0.8)
    out_path = 'UCSF_longitudinal_adj/' + i
    adjacency.to_csv(out_path)

### get patient CDR score

In [12]:
cdr = pd.read_csv("dataset/CDR.csv")
cdr = cdr[cdr['Phase']=='ADNI1']
cdr = cdr[['RID','VISCODE','EXAMDATE','CDGLOBAL']]
cdr = cdr.sort_values(by=['RID','EXAMDATE'])

per_patient = cdr.pivot(index='RID', columns='VISCODE', values="CDGLOBAL")
per_patient.drop('uns1', axis=1, inplace=True)
per_patient = per_patient[['sc','f','m06','m12','m18','m24','m36','m48']]

# fill NA's row wise
per_patient = per_patient.T.fillna(method='ffill').T
per_patient.to_csv("ADNI1_CDR_diagnosis.csv")